In [ ]:
import holodeck as holo
from holodeck import plot
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm

### Get PTA frequencies

In [ ]:
fobs_cents, fobs_edges = holo.utils.pta_freqs()

### Build Semi Analytic Model

In [ ]:
sam = holo.sams.Semi_Analytic_Model()
hard = holo.hardening.Hard_GW()

### Calculate Characteristic Strain for 5 Loudest Sources

In [ ]:
NREALS = 30             # number of random realizations
SHAPE = None            # choose an int for fast debugging, or None for full shape
# SHAPE = 40              # choose an int for fast debugging, or None for full shape
NLOUDEST = 5

hc_ss, hc_bg, sspar, bgpar = sam.gwb(
    fobs_edges, hard, realize=NREALS, loudest=NLOUDEST, params=True)

# hc_ss, hc_bg, = sam.gwb(
#     fobs_edges, hard, realize=NREALS, loudest=5, params=False)

### Plot Characteristic Strain for 5 Loudest Sources

In [ ]:
fig, ax = plot.figax(xlabel='Frequency (yr)', ylabel='Characteristic Strain, $h_c$')
xx = fobs_cents*YR

# plot power law for comparison
yy = xx**(-2/3)*1.1*10**-15
ax.plot(xx,yy, color='k', alpha=0.25, linestyle='dashed')

# plot BG median and CI
ax.plot(xx, np.median(hc_bg, axis=-1), color='k')
for pp in [50,95]:
    lo, hi = np.percentile(hc_bg, (50-pp/2, 50+pp/2), axis=-1)
    ax.fill_between(xx, lo, hi, color='k', alpha=0.2)

nreals = 5      # how many realizations to show
nloudest = 3    # how many loudest to show for each realization
colors=cm.rainbow(np.linspace(0,1,nreals))

rands = np.random.choice(np.arange(0,len(hc_ss[0,0])), 
                         size=nreals, replace=False, )
for rr, rand in enumerate(rands):
    cc = colors[rr] # get realization color

    ax.plot(xx, hc_bg[:,rand], alpha=0.5, zorder=6) # plot BG realization

    ax.scatter(xx, hc_ss[:,rand,0], marker='o', color=cc, alpha=0.2,
               edgecolors='k', zorder=5) # plot single loudest SS with black edges
    for ll in range(nloudest): # plot all other loudest SS
        ax.scatter(xx, hc_ss[:,rand,ll], marker='o', color=cc, alpha=0.2,
                   )

### Plot binary properties
If you want, you can also plot the parameters

* sspar gives [total mass, mass ratio, initial redshift, final redshift]
* bgpar gives hc_bg-weighted averages of 
    [total mass, mass ratio, initial redshift, final redshift,
     final comoving distance, final separation, final angular separation]


get the rest of single source parameters using
sspar = holo.single_sources.all_sspars(fobs_cents, sspar)

In [ ]:
total_mass = sspar[0] # shape [NFREQS, NREALS, NLOUDEST,]
mass_ratio = sspar[1]
initial_redshift = sspar[2]
final_redshift = sspar[3]

In [ ]:
fig = plot.plot_pars(fobs_cents, sspar, bgpar) # here's a mediocre plotting function I've already written

# Create Healpix Map using many loud single sources

### Function in the anisotropy.py module not yet pushed to NANOGrav

In [ ]:
def healpix_map(hc_ss, hc_bg, nside=8, seed=None, ret_seed=False):
    """ Build mollview array of hc^2/dOmega for a healpix map
    
    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of h_c^2/dOmega at every pixel for a mollview healpix map.
    
    NOTE: Could speed up the for-loops, but it's ok for now.
    """

    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    nfreqs = len(hc_ss)
    nreals = len(hc_ss[0])
    nloudest = len(hc_ss[0,0])

    # set random seed
    if seed is None:
        seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
    np.random.seed(seed)   

    # spread background evenly across pixels in moll_hc
    moll_hc = np.ones((nfreqs,nreals,npix)) * hc_bg[:,:,np.newaxis]**2/(npix*area) # (frequency, realization, pixel)

    # choose random pixels to place the single sources
    pix_ss = np.random.randint(0, npix-1, size=nfreqs*nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    for ff in range(nfreqs):
        for rr in range(nreals):
            for ll in range(nloudest):
                moll_hc[ff,rr,pix_ss[ff,rr,ll]] = (moll_hc[ff,rr,pix_ss[ff,rr,ll]] + hc_ss[ff,rr,ll]**2/area)
    if ret_seed:
        return moll_hc, seed           
    return moll_hc

### Calculate hc_ss for a larger number of single sources, for a more detailed map

In [ ]:
NREALS = 10             # number of random realizations, fewer for speed since we'll only use one at a time for a map
# SHAPE = None            # choose an int for fast debugging, or None for full shape
SHAPE = 40              # choose an int for fast debugging, or None for full shape
NLOUDEST = 100          # number of loudest single sources, more for a detailed map
hc_ss, hc_bg = sam.gwb(
    fobs_edges, hard, realize=NREALS, loudest=NLOUDEST, params=False)

### Make healpy map from single sources

In [ ]:
NSIDE = 16 # choose map resolution, which should be a power of 2
moll_hc = healpix_map(hc_ss, hc_bg, nside=NSIDE)
print(moll_hc.shape)

### Plot healpy map

In [ ]:
ff = 0                              # choose lowest frequency bin
rr = np.random.randint(0, NREALS)   # choose random realization
print(rr)
hp.mollview(moll_hc[ff,rr])         # healpix map

### Plot smoothed healpy map

In [ ]:
seed=None
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")     # print it out so we can reuse it if desired
np.random.seed(seed)  

deg = 30 # choose smoothing between ~0 and 180 degrees
smooth_map = hp.smoothing(moll_hc[ff,rr], fwhm=np.radians(deg))
fig,ax = plot.figax()
hp.mollview(smooth_map, hold=True, 
                title='Smoothed with fwhm=%d$\deg$' % (deg),
                cbar=False
                #min = 4e-33, max=1.5e-31, cbar=False
                )
